# clanker robot arm

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time
from collections import deque


### constants initialization

In [ ]:
SERIAL_PORT = 'COM3'
BAUD_RATE = 115200
CAMERA_INDEX = 0
SMOOTHING_WINDOW = 7 
